In [17]:
from dotenv import load_dotenv
import os, sys
import openai

import json, ndjson, pprint

from time import sleep
import logging
import datetime

In [18]:
# APIキーをファイルから読み込み，APIキー認証
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [19]:
# 処理時間を計測するため、loggerのフォーマット設定しています
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
format = '%(asctime)s [%(levelname)s] %(filename)s, lines %(lineno)d. %(message)s'
formatter = logging.Formatter(format, '%Y-%m-%d %H:%M:%S')
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [20]:
# トークログ(プロンプト)のバックアップログを取る関数
def AddBackUpLog_gptPrompt(prompt:dict):
    dt_now = datetime.datetime.now()
    BACKUP_LOG_PATH = f"./log/openai.ChatCompletion.create_{dt_now.year}-{dt_now.month}-{dt_now.day}.ndjson"
    with open(BACKUP_LOG_PATH, 'a') as f:
        writer = ndjson.writer(f, ensure_ascii=False)
        writer.writerow(prompt)

# 
def ChatCompletion(prompt:str, max_token:int = 4000):
    AddBackUpLog_gptPrompt({"role": "user", "content": prompt}) # GPT入力/出力のログを取る
    retry_interval = 2
    tries = 4
    for num_try in range(1, tries+1):
        try:
            print(f"START ChatCompletion.create Contents ... ", end="")
            response = openai.ChatCompletion.create(
                model = "gpt-3.5-turbo",
                messages = [{"role": "user", "content": prompt}],
                max_tokens = max_token
            )
            print(f"FINISH ChatCompletion.create")
            AddBackUpLog_gptPrompt(response["choices"][0]["message"]) # GPT入力/出力のログを取る
            break
        except:
            print(f"ChatCompletion.create ERROR")
            if num_try == tries:
                raise
            sleep(retry_interval)
            logger.info(f'リトライ回数:{num_try}回目')
            continue
    return response

In [21]:
prompt = "AIとはなんですか？"

res = ChatCompletion(prompt, max_token=200)
pprint.pprint(res)

START ChatCompletion.create Contents ... FINISH ChatCompletion.create
{'choices': [{'finish_reason': 'length',
              'index': 0,
              'message': {'content': 'AI（人工知能）は、コンピューターシステムやプログラムが人間の知能の一部を模倣することで、知識の獲得や推論、問題解決、学習、意思決定などのタスクを実行する能力を指します。AIは、機械学習、深層学習、自然言語処理、画像認識、音声認識などの技術を用いて実現されます。また、AIは様々な分野で活用され、自動運転、医療診断、音声アシスタント、金融',
                          'role': 'assistant'}}],
 'created': 1691926872,
 'id': 'chatcmpl-7n3l230FhvFqo2FfAnEZ0RnuEc9uq',
 'model': 'gpt-3.5-turbo-0613',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 199,
           'prompt_tokens': 15,
           'total_tokens': 214}}
